# Entanglement Distillation

## Initialization

In [1]:
import sympy as sp
import numpy as np

In [2]:
# Pauli matrices
E = sp.Matrix([[1,0],
               [0,1]])
X = sp.Matrix([[0,1],
               [1,0]])
Y = sp.Matrix([[0,-sp.I],
               [sp.I,0]])
Z = sp.Matrix([[1,0],
               [0,-1]])

# Rotation matrices
# type greek letters as e.g. \phi, then press tab
#ϕ_x,ϕ_y,ϕ_z = sp.symbols("ϕ_x,ϕ_y,ϕ_z")
def R_x(ϕ_x):
    return sp.exp(sp.I*ϕ_x*X/2).simplify()
def R_y(ϕ_y): 
    return sp.exp(sp.I*ϕ_y*Y/2).simplify()
def R_z(ϕ_z): 
    return sp.exp(sp.I*ϕ_z*Z/2).simplify()

U_BellToProd = sp.Matrix([[1,0,1,0],[0,1,0,1],[0,1,0,-1],[1,0,-1,0]])/sp.sqrt(2)
U_ProdToBell = U_BellToProd.H  # hermitian adjoint with .H

#ρ_1 = sp.MatrixSymbol("ρ_1",4,4)
#ρ_2 = sp.MatrixSymbol("ρ_2",4,4)
def Fidelity(ρ_1,ρ_2): 
    return sp.Trace(sp.sqrt(sp.sqrt(ρ_1)*ρ_2*sp.sqrt(ρ_1)))**2

In [3]:
# sympy allows latex style printing 
θ = sp.symbols("θ")
R_z(θ).simplify()

Matrix([
[exp(I*θ/2),           0],
[         0, exp(-I*θ/2)]])

## Entanglement Distillation Protocol

We will switch between the Bell basis and the product basis several times during the upcoming protocol.

Consider e.g. the Bell state $|{\phi_+}\rangle$:

In [4]:
ϕ_00 = U_BellToProd[:,0]  # Bell state (in product basis) can be read 
                          # out of basis change matrix
ρ_Bell_pb = ϕ_00*ϕ_00.H   # take outer product to obtain corresponding operator
ρ_Bell_pb

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [5]:
# Basis change using the unitary basis change matrices
ρ_bb = U_ProdToBell*ρ_pb*U_ProdToBell.H
ρ_bb

NameError: name 'ρ_pb' is not defined

Unitaries act by "sandwiching" around operator. $\rho$ in the Bell basis is diagonal and rank 1.

In [6]:
### Step 1: Choosing an initial state

In [7]:
# singlet bell state
ρ_Singlet = 1/2*sp.Matrix([[0, 0,  0, 0], 
                           [0, 1, -1, 0], 
                           [0, -1, 1, 0], 
                           [0, 0,  0, 0]])

# initial state
ρ_1 = sp.Matrix(
          [[0,    0,    0, 0], 
           [0,  sp.Rational(1,3), -sp.Rational(1,4), 0], 
           [0, -sp.Rational(1,4),  sp.Rational(2,3), 0],
           [0,    0,    0, 0]]
)

ρ_1

Matrix([
[0,    0,    0, 0],
[0,  1/3, -1/4, 0],
[0, -1/4,  2/3, 0],
[0,    0,    0, 0]])

In [8]:
# Compute the fidelity
Fidelity(ρ_Singlet,ρ_1).simplify()

0.750000000000000

Greater than 1/2 $\implies$ Can be distilled by our protocol

### Diagonalizing the state

First, consider a general density matrix

In [9]:
ρ = sp.MatrixSymbol("ρ",4,4)
sp.Matrix(ρ)

Matrix([
[ρ[0, 0], ρ[0, 1], ρ[0, 2], ρ[0, 3]],
[ρ[1, 0], ρ[1, 1], ρ[1, 2], ρ[1, 3]],
[ρ[2, 0], ρ[2, 1], ρ[2, 2], ρ[2, 3]],
[ρ[3, 0], ρ[3, 1], ρ[3, 2], ρ[3, 3]]])

Now act on both subsystems with the same randomly chosen Pauli operator,
e.g. $X\otimes X$.  
Recall that identity also is a Pauli operator.  

In [10]:
from sympy.physics.quantum.tensorproduct import TensorProduct,matrix_tensor_product
XX = TensorProduct(X,X)
YY = TensorProduct(Y,Y)
ZZ = TensorProduct(Z,Z)
ρ_new = (ρ + XX*ρ*XX + YY*ρ*YY + ZZ*ρ*ZZ)/4
sp.Matrix(ρ_new)

Matrix([
[ρ[0, 0]/2 + ρ[3, 3]/2,                     0,                     0, ρ[0, 3]/2 + ρ[3, 0]/2],
[                    0, ρ[1, 1]/2 + ρ[2, 2]/2, ρ[1, 2]/2 + ρ[2, 1]/2,                     0],
[                    0, ρ[1, 2]/2 + ρ[2, 1]/2, ρ[1, 1]/2 + ρ[2, 2]/2,                     0],
[ρ[0, 3]/2 + ρ[3, 0]/2,                     0,                     0, ρ[0, 0]/2 + ρ[3, 3]/2]])

The resulting matrix looks slightly suspicious.  
Maybe it is even simpler in another basis ...

In [11]:
ρ_new_bb = U_ProdToBell*ρ_new*U_ProdToBell.H
sp.simplify(sp.Matrix(ρ_new_bb))

Matrix([
[ρ[0, 0]/2 + ρ[0, 3]/2 + ρ[3, 0]/2 + ρ[3, 3]/2,                                             0,                                             0,                                             0],
[                                            0, ρ[1, 1]/2 + ρ[1, 2]/2 + ρ[2, 1]/2 + ρ[2, 2]/2,                                             0,                                             0],
[                                            0,                                             0, ρ[0, 0]/2 - ρ[0, 3]/2 - ρ[3, 0]/2 + ρ[3, 3]/2,                                             0],
[                                            0,                                             0,                                             0, ρ[1, 1]/2 - ρ[1, 2]/2 - ρ[2, 1]/2 + ρ[2, 2]/2]])

Indeed, in the Bell basis the matrix is diagonal.

Randomly choose one of three rotations (by 90°), apply it to ρ_new bilaterally:

In [12]:
φ = sp.pi/2

R_XX = matrix_tensor_product(R_x(φ),R_x(φ))
R_YY = matrix_tensor_product(R_y(φ),R_y(φ))
R_ZZ = matrix_tensor_product(R_z(φ),R_z(φ))
#ρ_new = sp.Matrix(ρ_new)
ρ_new2 = (R_XX*ρ_new*R_XX.H + R_YY*ρ_new*R_YY.H + R_ZZ*ρ_new*R_ZZ.H)/3
sp.simplify(sp.Matrix(ρ_new2))

Matrix([
[ρ[0, 0]/3 + ρ[1, 1]/6 + ρ[1, 2]/6 + ρ[2, 1]/6 + ρ[2, 2]/6 + ρ[3, 3]/3,                                                                     0,                                                                     0,                                                                     0],
[                                                                    0, ρ[0, 0]/6 + ρ[1, 1]/3 - ρ[1, 2]/6 - ρ[2, 1]/6 + ρ[2, 2]/3 + ρ[3, 3]/6, ρ[0, 0]/6 - ρ[1, 1]/6 + ρ[1, 2]/3 + ρ[2, 1]/3 - ρ[2, 2]/6 + ρ[3, 3]/6,                                                                     0],
[                                                                    0, ρ[0, 0]/6 - ρ[1, 1]/6 + ρ[1, 2]/3 + ρ[2, 1]/3 - ρ[2, 2]/6 + ρ[3, 3]/6, ρ[0, 0]/6 + ρ[1, 1]/3 - ρ[1, 2]/6 - ρ[2, 1]/6 + ρ[2, 2]/3 + ρ[3, 3]/6,                                                                     0],
[                                                                    0,                                                              

In [13]:
# in Bell basis
ρ_new2_bb = U_ProdToBell*ρ_new2*U_ProdToBell.H
sp.simplify(sp.Matrix(ρ_new2_bb))

Matrix([
[ρ[0, 0]/3 + ρ[1, 1]/6 + ρ[1, 2]/6 + ρ[2, 1]/6 + ρ[2, 2]/6 + ρ[3, 3]/3,                                                                     0,                                                                     0,                                             0],
[                                                                    0, ρ[0, 0]/3 + ρ[1, 1]/6 + ρ[1, 2]/6 + ρ[2, 1]/6 + ρ[2, 2]/6 + ρ[3, 3]/3,                                                                     0,                                             0],
[                                                                    0,                                                                     0, ρ[0, 0]/3 + ρ[1, 1]/6 + ρ[1, 2]/6 + ρ[2, 1]/6 + ρ[2, 2]/6 + ρ[3, 3]/3,                                             0],
[                                                                    0,                                                                     0,                                                               

Now, determine $\mathrm{tr}(\rho)$

In [14]:
sp.simplify(sp.trace(sp.Matrix(ρ_new2_bb)))

ρ[0, 0] + ρ[1, 1] + ρ[2, 2] + ρ[3, 3]

In [15]:
# Also fidelity = 
sp.simplify(Fidelity(sp.Matrix(ρ_new2),ρ_Singlet))

0.5*ρ[1, 1] - 0.5*ρ[1, 2] - 0.5*ρ[2, 1] + 0.5*ρ[2, 2]

enforce $\mathrm{tr}(\rho)=1$ by substituting $\rho_{00} \mapsto 1-\rho_{11}-\rho_{22}-\rho_{33}$  
Define variable $F$ and substitute:
$\rho_{11} \mapsto 2F+\rho_{12}-\rho_{21}-\rho_{22}$

In [16]:
F = sp.symbols("F")
ρ_new3_bb = sp.simplify(sp.Matrix(ρ_new2_bb).subs(ρ[0,0],1-ρ[1,1]-ρ[2,2]-ρ[3,3]))
ρ_new3_bb = sp.simplify(ρ_new3_bb.subs(ρ[1,1],2*F +ρ[1,2]+ρ[2,1]-ρ[2,2]))
sp.simplify(sp.Matrix(ρ_new3_bb))

Matrix([
[1/3 - F/3,         0,         0, 0],
[        0, 1/3 - F/3,         0, 0],
[        0,         0, 1/3 - F/3, 0],
[        0,         0,         0, F]])

The state above is the Werner state, and forms the basis for the rest of the distillation protocol

In [17]:
ρ_W_bb = sp.simplify(sp.Matrix(ρ_new3_bb))
ρ_W_pb = U_BellToProd*ρ_W_bb*U_BellToProd.H
ρ_W_pb

Matrix([
[1/3 - F/3,           0,           0,         0],
[        0,   F/3 + 1/6, 1/6 - 2*F/3,         0],
[        0, 1/6 - 2*F/3,   F/3 + 1/6,         0],
[        0,           0,           0, 1/3 - F/3]])

We apply a $Y$-gate on only one of the subsystems, which causes $|\psi_-\rangle$ and $|\phi_+\rangle$ to switch:

In [18]:
ρ_W2_pb = matrix_tensor_product(E,Y)*ρ_W_pb*matrix_tensor_product(E,Y)
ρ_W2_pb

Matrix([
[  F/3 + 1/6,         0,         0, 2*F/3 - 1/6],
[          0, 1/3 - F/3,         0,           0],
[          0,         0, 1/3 - F/3,           0],
[2*F/3 - 1/6,         0,         0,   F/3 + 1/6]])

The new state has now a high fidelity to $|\phi_+\rangle$

In [19]:
sp.simplify(Fidelity(ρ_Bell_pb,ρ_W2_pb))

F

Now consider the unitary $U_{ij} = |0\rangle\langle 0 |\otimes X_j+|1\rangle\langle 1 |\otimes \mathbb{1}_j$

Alice and Bob apply this unitary on two copies of their state within their 
local subsystem. It is here formally easier to describe their joint application
of the unitary, than to construct it from the individual unitaries.

In [25]:
P_0 = sp.Matrix([[1,0],[0,0]])
P_1 = sp.Matrix([[0,0],[0,1]])
U = (matrix_tensor_product(P_0,E,X,E)+matrix_tensor_product(P_1,E,E,E))*(matrix_tensor_product(E,P_0,E,X)+matrix_tensor_product(E,P_1,E,E))
U

Matrix([
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [26]:
G = sp.symbols("G")
ρ_W2_pb_other = ρ_W2_pb.subs(F,G) # Due to an issue with tensor products
# of symbolic matrices, they need to be given different parameters that 
# are identified later
ρ_W2_2copies_pb = matrix_tensor_product(sp.Matrix(ρ_W2_pb),sp.Matrix(ρ_W2_pb_other)) # here something is off
ρ_W2_2copies_pb = ρ_W2_2copies_pb.subs(G,F)
ρ_W3_2copies_pb = sp.simplify(U*ρ_W2_2copies_pb*U.H)
ρ_W3_2copies_pb

Matrix([
[       (2*F + 1)**2/36,                     0,                     0, (2*F + 1)*(4*F - 1)/36,            0,                     0,                     0,            0,            0,                     0,                     0,            0,        (4*F - 1)**2/36,                     0,                     0, (2*F + 1)*(4*F - 1)/36],
[                     0, -F**2/9 + F/18 + 1/18,                     0,                      0,            0,                     0,                     0,            0,            0,                     0,                     0,            0,                      0,                     0, -(F - 1)*(4*F - 1)/18,                      0],
[                     0,                     0, -F**2/9 + F/18 + 1/18,                      0,            0,                     0,                     0,            0,            0,                     0,                     0,            0,                      0, -(F - 1)*(4*F - 1)/18,                     0,   

### Making measurements

In [27]:
P_00 = matrix_tensor_product(E,E,P_0,P_0)
P_11 = matrix_tensor_product(E,E,P_1,P_1)

In [28]:
p = sp.simplify(sp.trace(P_00*ρ_W3_2copies_pb)+sp.trace(P_11*ρ_W3_2copies_pb))
p

8*F**2/9 - 4*F/9 + 5/9

In [34]:
ρ_reduced = (P_00*ρ_W3_2copies_pb*P_00)[0::4,0::4]
# dont forget to normalize, and leave out the partial trace

In [38]:
sp.simplify(Fidelity(ρ_Bell_pb,ρ_reduced))

5*F**2/9 - F/9 + 1/18